In [1134]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"",regex=True)

# Check names.
student_data_df.head(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
5,5,Bryan Miranda,M,9th,Huang High School,94,94
6,6,Sheena Carter,F,11th,Huang High School,82,80
7,7,Nicole Baker,F,12th,Huang High School,96,69
8,8,Michael Roth,M,10th,Huang High School,95,87
9,9,Matthew Greene,M,10th,Huang High School,96,84


## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

In [1135]:
# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

In [1136]:
# Step 2. Use the loc method on the student_data_df to select all the reading scores from the 9th grade at Thomas High School and replace them with NaN.
student_data_df.loc[((student_data_df['grade'] == '9th') & (student_data_df['school_name'] == 'Thomas High School')),'reading_score'] = np.nan

In [1137]:
#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
student_data_df.loc[((student_data_df['grade'] == '9th') & (student_data_df['school_name'] == 'Thomas High School')),'math_score'] = np.nan

In [1138]:
#  Step 4. Check the student data for NaN's. 
#student_data_df.loc[student_data_df["school_name"] == "Thomas High School"]
student_data_df.isnull().sum()

Student ID         0
student_name       0
gender             0
grade              0
school_name        0
reading_score    461
math_score       461
dtype: int64

## Deliverable 2 : Repeat the school district analysis

### District Summary

In [1139]:
# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])

In [1140]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()
# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

In [1141]:
# Calculate the Average Scores using the "clean_student_data".
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [1142]:
# Step 1. Get the number of students that are in ninth grade at Thomas High School.
# These students have no grades. 
THS_9th = school_data_complete_df.loc[((school_data_complete_df["school_name"]=="Thomas High School")
                                       &(school_data_complete_df["grade"]=="9th")),"Student ID"].count()

# Get the total student count 
pre_9NaN_student_count = school_data_complete_df["Student ID"].count()
# Step 2. Subtract the number of students that are in ninth grade at 
# Thomas High School from the total student count to get the new total student count.
student_count = pre_9NaN_student_count - THS_9th

In [1143]:
# Calculate the passing rates using the "clean_student_data".
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]

In [1144]:
# Step 3. Calculate the passing percentages with the new total student count.
passing_math_percentage = passing_math_count / float(student_count) * 100

passing_reading_percentage = passing_reading_count / float(student_count) * 100

In [1145]:
# Calculate the students who passed both reading and math.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                               & (school_data_complete_df["reading_score"] >= 70)]
# Calculate the number of students that passed both reading and math.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()
# Step 4.Calculate the overall passing percentage with new total student count.
overall_passing_percentage = overall_passing_math_reading_count / student_count * 100

In [1146]:
# Create a DataFrame
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students": student_count, 
          "Total Budget": total_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])



# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df.style.set_caption("Overall School District Summary")

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"38,709","$24,649,428.00",78.9,81.9,74.8,85.7,64.9


##  School Summary

In [1147]:
# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100
per_school_counts

Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: school_name, dtype: int64

In [1148]:
# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})
per_school_summary_df.loc["Thomas High School"]

School Type                Charter
Total Students                1635
Total School Budget      1043130.0
Per Student Budget           638.0
Average Math Score       83.350937
Average Reading Score    83.896082
% Passing Math           66.911315
% Passing Reading        69.663609
% Overall Passing        65.076453
Name: Thomas High School, dtype: object

In [1149]:
# Format the Total School Budget and the Per Student Budget
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [1150]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).
THS_minus9_count = school_data_complete_df.loc[(school_data_complete_df["school_name"]=="Thomas High School") &
                                                ((school_data_complete_df["grade"]=="10th") |
                                                (school_data_complete_df["grade"]=="11th") |
                                                (school_data_complete_df["grade"]=="12th")),"Student ID"].count()

In [1151]:
# Step 6. Get all the students passing math from THS
THS_passing_math = school_data_complete_df.loc[(school_data_complete_df["school_name"] == "Thomas High School") & 
                                                 (school_data_complete_df["math_score"] >= 70)]

In [1152]:
# Step 7. Get all the students passing reading from THS
THS_passing_reading = school_data_complete_df.loc[(school_data_complete_df["school_name"] == "Thomas High School") & 
                                                 (school_data_complete_df["reading_score"] >= 70)]

In [1153]:
# Step 8. Get all the students passing math and reading from THS
THS_passing_math_reading = school_data_complete_df.loc[(school_data_complete_df["school_name"] == "Thomas High School") & 
                                                       (school_data_complete_df["reading_score"] >= 70) &
                                                       (school_data_complete_df["math_score"] >= 70)]

In [1154]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School. 
THS_passing_math_percentage = len(THS_passing_math) / THS_minus9_count * 100

In [1155]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.
THS_passing_reading_percentage = len(THS_passing_reading) / THS_minus9_count * 100

In [1156]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 
THS_overall_passing_percentage = len(school_data_complete_df.loc[(school_data_complete_df["school_name"] == "Thomas High School") & 
                                                             (school_data_complete_df["reading_score"] >= 70) &
                                                             (school_data_complete_df["math_score"] >= 70)]) / THS_minus9_count * 100

In [1157]:
# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School","% Passing Math"] = THS_passing_math_percentage

In [1158]:
# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School","% Passing Reading"] = THS_passing_reading_percentage

In [1159]:
# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School","% Overall Passing"] = THS_overall_passing_percentage

## High and Low Performing Schools 

In [1160]:
# Sort and show top five schools.
per_school_summary_df.sort_values(["% Overall Passing"], ascending=False).head(5).style.set_caption("Summaries of the Top Five Performing Schools by Overall Passing Percentage")

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.350937,83.896082,93.185690,97.018739,90.630324
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [1161]:
# Sort and show bottom five schools.
per_school_summary_df.sort_values(["% Overall Passing"], ascending=False).tail(5).style.set_caption("Summaries of the Bottom Five Performing Schools by Overall Passing Percentage")

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247


## Math and Reading Scores by Grade

In [1162]:
# Create a Series of scores by grade levels using conditionals.
## Student comment - the above instructions specify a "Series", however the dataframes contain 2 separate series of scores that would be erroneous to combine. This is likely meant to be a "dataframe."
grade_9_scores = school_data_complete_df[school_data_complete_df["grade"]=="9th"]

grade_10_scores = school_data_complete_df[school_data_complete_df["grade"]=="10th"]

grade_11_scores = school_data_complete_df[school_data_complete_df["grade"]=="11th"]

grade_12_scores = school_data_complete_df[school_data_complete_df["grade"]=="12th"]

# Group each school Series by the school name for the average math score.
grade_9_avgMath = grade_9_scores.groupby(grade_9_scores["school_name"]).mean()["math_score"]

grade_10_avgMath = grade_10_scores.groupby(grade_10_scores["school_name"]).mean()["math_score"]

grade_11_avgMath = grade_11_scores.groupby(grade_11_scores["school_name"]).mean()["math_score"]

grade_12_avgMath = grade_12_scores.groupby(grade_12_scores["school_name"]).mean()["math_score"]

# Group each school Series by the school name for the average reading score.

grade_9_avgRead = grade_9_scores.groupby(grade_9_scores["school_name"]).mean()["reading_score"]

grade_10_avgRead = grade_10_scores.groupby(grade_10_scores["school_name"]).mean()["reading_score"]

grade_11_avgRead = grade_11_scores.groupby(grade_11_scores["school_name"]).mean()["reading_score"]

grade_12_avgRead = grade_12_scores.groupby(grade_12_scores["school_name"]).mean()["reading_score"]
  
# grade_12_avgRead.reset_index()

In [1163]:
# Combine each Series for average math scores by school into single data frame.

avg_math_by_school_grade = pd.DataFrame({
               "9th": grade_9_avgMath,
               "10th": grade_10_avgMath,
               "11th": grade_11_avgMath,
               "12th": grade_12_avgMath})
avg_math_by_school_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [1164]:
# Combine each Series for average reading scores by school into single data frame.
avg_read_by_school_grade = pd.DataFrame({
               "9th": grade_9_avgRead,
               "10th": grade_10_avgRead,
               "11th": grade_11_avgRead,
               "12th": grade_12_avgRead})
avg_read_by_school_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [1165]:
# Format each grade column.

# math
avg_math_by_school_grade["9th"] = avg_math_by_school_grade["9th"].map("{:.1f}".format)
avg_math_by_school_grade["10th"] = avg_math_by_school_grade["10th"].map("{:.1f}".format)
avg_math_by_school_grade["11th"] = avg_math_by_school_grade["11th"].map("{:.1f}".format)
avg_math_by_school_grade["12th"] = avg_math_by_school_grade["12th"].map("{:.1f}".format)

#reading
avg_read_by_school_grade["9th"] = avg_read_by_school_grade["9th"].map("{:.1f}".format)
avg_read_by_school_grade["10th"] = avg_read_by_school_grade["10th"].map("{:.1f}".format)
avg_read_by_school_grade["11th"] = avg_read_by_school_grade["11th"].map("{:.1f}".format)
avg_read_by_school_grade["12th"] = avg_read_by_school_grade["12th"].map("{:.1f}".format)


In [1166]:
# Remove the index.
avg_math_by_school_grade.index.name = None

# Display the data frame
avg_math_by_school_grade.style.set_caption("Average Math Scores By School, By Grade")

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [1167]:
## Remove the index.
avg_read_by_school_grade.index.name = None

# Display the data frame
avg_read_by_school_grade.style.set_caption("Average Reading Scores By School, By Grade")

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


## Scores by School Spending

In [1168]:
# Establish the spending bins and group names.
spending_bins = [0, 585, 615, 645, 675]
pd.cut(per_school_capita, spending_bins)
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

# Categorize spending based on the bins.
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)
#per_school_summary_df

In [1169]:
# Calculate averages for the desired columns. 
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [1170]:
# Create the DataFrame
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

In [1171]:
# Format the DataFrame 
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)

spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)

spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)

spending_summary_df.style.set_caption("Scores by School Spending")

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,83.6,83.9,94,96,90
$630-644,79.1,81.9,76,86,66
$645-675,77.0,81.0,66,81,54


## Scores by School Size

In [1172]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


#per_school_summary_df["Total Students"].describe()
# student_count_per_school = per_school_summary_df["Total Students"]
# student_count_per_school
# school_size_bins = [0,1500, 2000,3750,5000]
# school_size_bin_names = ["<1500","1500-2000","2000-3750","3750-5000"]
    # student_count_per_school.groupby(pd.cut(student_count_per_school,school_size_bins)).count()

# Categorize spending based on the bins.
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

In [1173]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]

size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]

size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]

size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]


In [1174]:
# Assemble into DataFrame. 
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})

In [1175]:
# Format the DataFrame  
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)

size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df.style.set_caption("Scores by School Size")

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,94,97,91
Large (2000-5000),77.7,81.3,70,83,58


## Scores by School Type

In [1176]:
# Calculate averages for the desired columns. 
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]

type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]

type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]

type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

In [1177]:
# Assemble into DataFrame. 
type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})

In [1178]:
# # Format the DataFrame 
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df.style.set_caption("Scores by School Type")

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54


In [1179]:
per_school_summary_df.loc["Thomas High School"]

School Type                                 Charter
Total Students                                 1635
Total School Budget                   $1,043,130.00
Per Student Budget                          $638.00
Average Math Score                        83.350937
Average Reading Score                     83.896082
% Passing Math                             93.18569
% Passing Reading                         97.018739
% Overall Passing                         90.630324
Spending Ranges (Per Student)              $630-644
School Size                      Medium (1000-2000)
Name: Thomas High School, dtype: object